In [0]:
from random import choice, random
from string import printable
from pyspark.sql.functions import current_timestamp

In [0]:
categories = (
    "discount",
    "home improvement",
    "outdoor",
    "lawn",
    "camping",
    "household",
    "sports",
    "utility",
    "maintenance",
    "repair",
    "cooking",
    "kitchen",
    "garage",
    "appliance",
    "software",
    "hardware",
    "electronics",
    "video games",
    "consoles",
    "tv",
    "entertainment",
    "gardening",
)

In [0]:
source_data = [
    (
        i, f"item{i}", i * 20, random(), choice(categories), choice(tuple(printable))
    ) for i in range(10000000)
]

In [0]:
source_df = spark.createDataFrame(source_data, [
    "id", "name", "inventory", "proportion_sold", "category", "abbreviation"
])

In [0]:
%sql
USE CATALOG dihpoc;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS testing;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS sink;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS bronze;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS silver;

In [0]:
source_df.withColumn(
    "record_timestamp", current_timestamp(),
).write.format("delta").saveAsTable(
    name="testing.source_table",
    mode="overwrite",
    overwriteSchema=True,
)

In [0]:
%sql
DROP TABLE IF EXISTS testing.test_table1;
CREATE TABLE testing.test_table1 (
  item_id BIGINT NOT NULL PRIMARY KEY,
  percent_sold VARCHAR(7) NOT NULL,
  record_timestamp TIMESTAMP NOT NULL
)

In [0]:
%sql
DROP TABLE IF EXISTS sink.test_table2;
CREATE TABLE sink.test_table2 (
  id BIGINT NOT NULL PRIMARY KEY,
  total_sales BIGINT NOT NULL,
  item_full_name STRING NOT NULL,
  created_timestamp TIMESTAMP NOT NULL
)

In [0]:
dbutils.fs.mkdirs("/dbfs/testing/")

In [0]:
dbutils.fs.mkdirs("/dbfs/testing/checkpoint_tests/")

In [0]:
dbutils.fs.rm("/dbfs/testing/checkpoint_tests/", True)

In [0]:
df=(
    spark.read.csv(
        path="abfss://landing@sbyccdevemdplandsa.dfs.core.windows.net/sobeys/merchandising/ecc/MARA/2023/02/17",
        header=True,
    )
)
display(df)